In [2]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import seaborn as sns

In [10]:
read_file_lst = glob.glob('C:/Users/yuns/Desktop/study/arena/read/read_test/*')

In [11]:
exclude_file_lst = ['read.tar']

In [12]:
read_df_lst = []
for f in read_file_lst:
    file_name = os.path.basename(f)
    if file_name in exclude_file_lst:
        print(file_name)
    else:
        df_temp = pd.read_csv(f, header=None, names=['raw'])
        df_temp['dt'] = file_name[:8]
        df_temp['hr'] = file_name[8:10]
        df_temp['user_id'] = df_temp['raw'].str.split(' ').str[0]
        df_temp['article_id'] = df_temp['raw'].str.split(' ').str[1:].str.join(' ').str.strip()
        read_df_lst.append(df_temp)

In [13]:
read = pd.concat(read_df_lst)

In [14]:
read.head()

,raw,dt,hr,user_id,article_id
0,#e208be4ffea19b1ceb5cea2e3c4dc32c @kty0613_91,20181001,00,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91
1,#0a3d493f3b2318be80f391eaa00bfd1c @miamiyoung_...,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@miamiyoung_31 @banksalad_49 @rlfrjsdn_95 @rea...
2,#b90d3ee7ed0d7d827aae168e159749f1 @joeunha_4 @...,20181001,00,#b90d3ee7ed0d7d827aae168e159749f1,@joeunha_4 @yoonvi_3
3,#b8b9d09fe2961fd62edc94912bf75a90 @hyejinchoi_...,20181001,00,#b8b9d09fe2961fd62edc94912bf75a90,@hyejinchoi_122 @hyejinchoi_86 @hyejinchoi_42 ...
4,#072f742eda9359cdac03ad080193c11d @doyeonsunim...,20181001,00,#072f742eda9359cdac03ad080193c11d,@doyeonsunim_240 @k52524_297 @bule13_33 @kwong...


In [15]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(' ')))

In [16]:
read_cnt_by_user = read['article_id'].str.split(' ').map(len)

In [17]:
read_raw = pd.DataFrame({'dt': np.repeat(read['dt'], read_cnt_by_user),
                         'hr': np.repeat(read['hr'], read_cnt_by_user),
                         'user_id': np.repeat(read['user_id'], read_cnt_by_user),
                         'article_id': chainer(read['article_id'])})

In [18]:
read_raw.head()

,dt,hr,user_id,article_id
0,20181001,00,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@miamiyoung_31
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@banksalad_49
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@rlfrjsdn_95
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@readme999_140


In [19]:
writers = read_raw['article_id'].str.split('_').str[0]

In [20]:
read_raw['writer'] = writers

In [21]:
read_raw.head()

,dt,hr,user_id,article_id,writer
0,20181001,00,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91,@kty0613
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@miamiyoung_31,@miamiyoung
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@banksalad_49,@banksalad
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@rlfrjsdn_95,@rlfrjsdn
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@readme999_140,@readme999


In [22]:
read_raw.groupby(['user_id','writer']).count()

dt  hr  article_id
user_id                           writer                              
#0000fdba8f35c76eacab74c5c6bc7f1a @lazypic           2   2           2
#00050506dfe68410df8f26ca57bcce1e @tigeryoonz        1   1           1
                                  @woobinnada        2   2           2
#000565ae879e313dbf7374fd0f7a59e9 @koreatrail        1   1           1
#00096e2aba749c6ae47baa82d1a6d991 @yeonjuni          1   1           1
#000fe61478d384d09f3bcdd0c2f5227d @varo              2   2           2
#0018e6ffeffe1c60064a1ebf21e5708b @youngjishin       2   2           2
#001b05975c5fbf6232058979294965f2 @miamiyoung        2   2           2
#001e5d07debb1a377dac6463291b7f87 @withjin10         1   1           1
#002299f5e10111c19ac32d432ba03371 @miamiyoung        4   4           4
                                  @sdain-sygang      1   1           1
#0032e042716c8a4cec2e0fda00c32965 @davidinlund      11  11          11
                                  @rhodeslaw         2   2           2
#0033ecfdcf5f7053bf54a3d0ffc4ec1e @dooropener        1   1           1
                                  @mingamsungfilm    1   1           1
                                  @pjy6114           1   1           1
                                  @rhkrwndgml        1   1           1
                                  @wip7777           1   1           1
#003410c95e33d608695b3f68bffd09fe @andycha           1   1           1
                                  @bigaid            1   1           1
                                  @brandbooster      1   1           1
                                  @clairep           1   1           1
                                  @hifiaudio         1   1           1
                                  @joohyoang         1   1           1
                                  @lovebrander       1   1           1
                                  @okwinus           1   1           1
                                  @plusclov          1   1           1
                                  @scandilife        1   1           1
                                  @sktjdghks         3   3           3
                                  @sosoceo           2   2           2
...                                                 ..  ..         ...
#fff58a923e4c956cde43495f8e3c285a @crystalclear      2   2           2
                                  @dailybotong       3   3           3
                                  @dailylife         6   6           6
                                  @dooropener        2   2           2
                                  @greendigital      1   1           1
                                  @jawssss           2   2           2
                                  @jooyoon           2   2           2
                                  @lamuse1           2   2           2
                                  @ljg523            2   2           2
                                  @lovebrander       2   2           2
                                  @ltmgn             2   2           2
                                  @mad9335           1   1           1
                                  @milkybaby4u       4   4           4
                                  @mumyung           3   3           3
                                  @neverupneverin    2   2           2
                                  @shoong810         1   1           1
                                  @simu-look         2   2           2
                                  @stars             2   2           2
                                  @whddhr9305        2   2           2
                                  @zenerkrepresent   1   1           1
#fff6b3e9c7cfd3876fd2b8a4dfd08af0 @allstay           1   1           1
                                  @jjanggutwin       8   8           8
#fff6fc1623cd1b2e51df1b2996eff69b @dryjshin          1   1           1
#fffb1351938a67654fa921cb47750e72 @alexkang          2   2           2
     

In [38]:
del read_raw['dt']

In [39]:
del read_raw['hr']

In [40]:
read_raw.head()

,user_id,article_id,writer
0,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91,@kty0613
1,#0a3d493f3b2318be80f391eaa00bfd1c,@miamiyoung_31,@miamiyoung
1,#0a3d493f3b2318be80f391eaa00bfd1c,@banksalad_49,@banksalad
1,#0a3d493f3b2318be80f391eaa00bfd1c,@rlfrjsdn_95,@rlfrjsdn
1,#0a3d493f3b2318be80f391eaa00bfd1c,@readme999_140,@readme999


In [23]:
user_writer_count = read_raw.groupby(['user_id','writer']).count()

In [24]:
user_writer_count.head()

dt  hr  article_id
user_id                           writer                         
#0000fdba8f35c76eacab74c5c6bc7f1a @lazypic      2   2           2
#00050506dfe68410df8f26ca57bcce1e @tigeryoonz   1   1           1
                                  @woobinnada   2   2           2
#000565ae879e313dbf7374fd0f7a59e9 @koreatrail   1   1           1
#00096e2aba749c6ae47baa82d1a6d991 @yeonjuni     1   1           1

In [33]:
path = "C:/Users/yuns/Desktop/study/arena/read/metadata.json"
meatadata = pd.read_json(path, lines=True)

In [36]:
metadata = meatadata

In [37]:
metadata.head()

,article_id,display_url,id,keyword_list,magazine_id,reg_ts,sub_title,title,user_id
0,782,https://brunch.co.kr/@bookdb/782,@bookdb_782,"[여행, 호주, 국립공원]",8982,1474944427000,세상 어디에도 없는 호주 Top 10,"사진으로 옮기기에도 아까운, 리치필드 국립공원",@bookdb
1,81,https://brunch.co.kr/@kohwang56/81,@kohwang56_81,"[목련꽃, 아지랑이, 동행]",12081,1463092749000,,[시] 서러운 봄,@kohwang56
2,4,https://brunch.co.kr/@hannahajink/4,@hannahajink_4,[],0,1447997287000,무엇 때문에,무엇을 위해,@hannahajink
3,88,https://brunch.co.kr/@bryceandjuli/88,@bryceandjuli_88,"[감정, 마음, 위로]",16315,1491055161000,,싫다,@bryceandjuli
4,34,https://brunch.co.kr/@mijeongpark/34,@mijeongpark_34,"[유럽여행, 더블린, 아일랜드]",29363,1523292942000,#7. 내 친구의 집은 어디인가,Dubliner#7,@mijeongpark


In [38]:
read_raw.head()

,dt,hr,user_id,article_id,writer
0,20181001,00,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91,@kty0613
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@miamiyoung_31,@miamiyoung
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@banksalad_49,@banksalad
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@rlfrjsdn_95,@rlfrjsdn
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@readme999_140,@readme999


In [45]:
metadata['article_id'] = metadata['id']

In [46]:
metadata.head()

,article_id,display_url,id,keyword_list,magazine_id,reg_ts,sub_title,title,user_id
0,@bookdb_782,https://brunch.co.kr/@bookdb/782,@bookdb_782,"[여행, 호주, 국립공원]",8982,1474944427000,세상 어디에도 없는 호주 Top 10,"사진으로 옮기기에도 아까운, 리치필드 국립공원",@bookdb
1,@kohwang56_81,https://brunch.co.kr/@kohwang56/81,@kohwang56_81,"[목련꽃, 아지랑이, 동행]",12081,1463092749000,,[시] 서러운 봄,@kohwang56
2,@hannahajink_4,https://brunch.co.kr/@hannahajink/4,@hannahajink_4,[],0,1447997287000,무엇 때문에,무엇을 위해,@hannahajink
3,@bryceandjuli_88,https://brunch.co.kr/@bryceandjuli/88,@bryceandjuli_88,"[감정, 마음, 위로]",16315,1491055161000,,싫다,@bryceandjuli
4,@mijeongpark_34,https://brunch.co.kr/@mijeongpark/34,@mijeongpark_34,"[유럽여행, 더블린, 아일랜드]",29363,1523292942000,#7. 내 친구의 집은 어디인가,Dubliner#7,@mijeongpark


In [48]:
read_meta = pd.merge(read_raw, metadata, on="article_id")

In [55]:
read_meta.head()

,dt,hr,user_id_x,article_id,writer,keyword_list,sub_title,title,user_id_y
0,20181001,00,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91,@kty0613,"[연애, 남자친구, 이성]",문제는 만남의 '기회'가 아닌 만남의 '기준',좋은 이성을 만나는 기준,@kty0613
1,20181001,00,#e6097910257c8b79380da50b4bce705b,@kty0613_91,@kty0613,"[연애, 남자친구, 이성]",문제는 만남의 '기회'가 아닌 만남의 '기준',좋은 이성을 만나는 기준,@kty0613
2,20181001,00,#abfa7b43a9c3f4894ca450b0436fcd38,@kty0613_91,@kty0613,"[연애, 남자친구, 이성]",문제는 만남의 '기회'가 아닌 만남의 '기준',좋은 이성을 만나는 기준,@kty0613
3,20181001,00,#abfa7b43a9c3f4894ca450b0436fcd38,@kty0613_91,@kty0613,"[연애, 남자친구, 이성]",문제는 만남의 '기회'가 아닌 만남의 '기준',좋은 이성을 만나는 기준,@kty0613
4,20181001,00,#bd4b6d5a32bcbe4a81419309af898630,@kty0613_91,@kty0613,"[연애, 남자친구, 이성]",문제는 만남의 '기회'가 아닌 만남의 '기준',좋은 이성을 만나는 기준,@kty0613


In [56]:
del read_meta['user_id_y']

In [57]:
read_meta['reader_id'] = read_meta['user_id_x']

In [58]:
read_meta.head() 

,dt,hr,user_id_x,article_id,writer,keyword_list,sub_title,title,reader_id
0,20181001,00,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91,@kty0613,"[연애, 남자친구, 이성]",문제는 만남의 '기회'가 아닌 만남의 '기준',좋은 이성을 만나는 기준,#e208be4ffea19b1ceb5cea2e3c4dc32c
1,20181001,00,#e6097910257c8b79380da50b4bce705b,@kty0613_91,@kty0613,"[연애, 남자친구, 이성]",문제는 만남의 '기회'가 아닌 만남의 '기준',좋은 이성을 만나는 기준,#e6097910257c8b79380da50b4bce705b
2,20181001,00,#abfa7b43a9c3f4894ca450b0436fcd38,@kty0613_91,@kty0613,"[연애, 남자친구, 이성]",문제는 만남의 '기회'가 아닌 만남의 '기준',좋은 이성을 만나는 기준,#abfa7b43a9c3f4894ca450b0436fcd38
3,20181001,00,#abfa7b43a9c3f4894ca450b0436fcd38,@kty0613_91,@kty0613,"[연애, 남자친구, 이성]",문제는 만남의 '기회'가 아닌 만남의 '기준',좋은 이성을 만나는 기준,#abfa7b43a9c3f4894ca450b0436fcd38
4,20181001,00,#bd4b6d5a32bcbe4a81419309af898630,@kty0613_91,@kty0613,"[연애, 남자친구, 이성]",문제는 만남의 '기회'가 아닌 만남의 '기준',좋은 이성을 만나는 기준,#bd4b6d5a32bcbe4a81419309af898630


In [103]:
cnt = read_meta['keyword_list'].map(len)

In [107]:
list(chain.from_iterable(read_meta['keyword_list']))

['연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자친구',
 '이성',
 '연애',
 '남자

In [108]:
read_raw = pd.DataFrame({'reader_id': np.repeat(read_meta['reader_id'], cnt),
                         'article_id': np.repeat(read_meta['article_id'], cnt),
                         'title': np.repeat(read_meta['title'], cnt),
                         'sub_title': np.repeat(read_meta['sub_title'], cnt),
                         'keyword': list(chain.from_iterable(read_meta['keyword_list']))})

In [109]:
read_raw.head()

,reader_id,article_id,title,sub_title,keyword
0,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91,좋은 이성을 만나는 기준,문제는 만남의 '기회'가 아닌 만남의 '기준',연애
0,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91,좋은 이성을 만나는 기준,문제는 만남의 '기회'가 아닌 만남의 '기준',남자친구
0,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91,좋은 이성을 만나는 기준,문제는 만남의 '기회'가 아닌 만남의 '기준',이성
1,#e6097910257c8b79380da50b4bce705b,@kty0613_91,좋은 이성을 만나는 기준,문제는 만남의 '기회'가 아닌 만남의 '기준',연애
1,#e6097910257c8b79380da50b4bce705b,@kty0613_91,좋은 이성을 만나는 기준,문제는 만남의 '기회'가 아닌 만남의 '기준',남자친구


In [114]:
read_raw.groupby(['reader_id','keyword']).count()

article_id  title  sub_title
reader_id                         keyword                              
#0000fdba8f35c76eacab74c5c6bc7f1a 그림에세이             2      2          2
                                  도쿄                2      2          2
                                  여행                2      2          2
#00050506dfe68410df8f26ca57bcce1e 동영상               1      1          1
                                  디자이너              1      1          1
                                  시퀀스               1      1          1
                                  영상편집              1      1          1
                                  예술                1      1          1
                                  전문가               1      1          1
                                  직업                1      1          1
                                  첫차                1      1          1
                                  프리미어              1      1          1
#000565ae879e313dbf7374fd0f7a59e9 시니어세대             1      1          1
                                  여행사이트             1      1          1
                                  체험여행              1      1          1
#001b05975c5fbf6232058979294965f2 가계부               2      2          2
                                  여행계획              2      2          2
                                  절약                2      2          2
#001e5d07debb1a377dac6463291b7f87 구곡폭포              1      1          1
                                  국내여행              1      1          1
                                  춘천                1      1          1
#002299f5e10111c19ac32d432ba03371 건강                1      1          1
                                  경영                2      2          2
                                  구조조정              2      2          2
                                  부자                1      1          1
                                  상품                1      1          1
                                  쇼핑                1      1          1
                                  양치질               1      1          1
                                  육아                1      1          1
                                  은행                2      2          2
...                                               ...    ...        ...
#fff58a923e4c956cde43495f8e3c285a 재즈                2      2          2
                                  지구                1      1          1
                                  총기                1      1          1
                                  취업                2      2          2
                                  커플                2      2          2
                                  패션                1      1          1
                                  포트폴리오             1      1          1
                                  하나                2      2          2
#fff6b3e9c7cfd3876fd2b8a4dfd08af0 가평펜션              1      1          1
                                  스파펜션              1      1          1
                                  스파펜션추천            1      1          1
#fff6fc1623cd1b2e51df1b2996eff69b 독서                1      1          1
                                  생각                1      1          1
#fffb1351938a67654fa921cb47750e72 UI                2      2          2
                                  UX                2      2          2
                                  강동원               2      2          2
                                  김지운               2      2          2
                                  디자인               2      2          2
                                  리뷰                2      2          2
                                  영상                2      2          2
                                  영화                4      4          4
                                  인랑                2      2          2
                                  하정우               2      2          2
#fffcc9c67c

In [ ]:
path = "C:/Users/yuns/Desktop/study/arena/read/contents/data.0"
contents_data = pd.read_json(path, lines=True)

In [ ]:
contents_data.head()

In [116]:
from gensim.models import Word2Vec

In [95]:
embedding_model = Word2Vec(tokenized_contents, size=100, window = 2, min_count=50, workers=4, iter=100, sg=1)

144755